<a href="https://colab.research.google.com/github/KorStats/ocr_receipt/blob/main/%EC%A6%9D%EB%B6%84/P%EB%B6%84%EB%A5%98%20%EC%A6%9D%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화  'Colab Notebooks'   images.zip        labels.zip   val.txt
 classes.txt	  images_1650.zip    labels_1650.zip  'My Drive'


In [ ]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [ ]:
import pandas as pd
df=pd.read_csv('/content/dataset/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [ ]:
df=df[(df['digit_1'] =='P')] #조건에 맞는 데이터 출력

In [ ]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000016,P,85,856,태권도장에서,학생및일반대사으로,태권도
1,id_0000028,P,85,856,사무실에서,직원들대상,교육관
2,id_0000054,P,85,855,학원,"학생, 일반인대상으로",중국어교육
3,id_0000140,P,85,856,사업장에서,일반고객대상으로,요가
4,id_0000143,P,85,856,학원에서,일반인대상으로,캘리그라피
...,...,...,...,...,...,...,...
46605,id_0999956,P,85,855,학원에서,학생을 대상으로,영어
46606,id_0999965,P,85,856,학원에서,일반인 대상으로,"캘리그라프,팝아트 등"
46607,id_0999966,P,85,855,학원에서,중고생을 대상으로,수학 교습
46608,id_0999990,P,85,855,공부방에서,학생대상,교과목


In [ ]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000016,P,85,856,태권도장에서 학생및일반대사으로 태권도
1,id_0000028,P,85,856,사무실에서 직원들대상 교육관
2,id_0000054,P,85,855,"학원 학생, 일반인대상으로 중국어교육"
3,id_0000140,P,85,856,사업장에서 일반고객대상으로 요가
4,id_0000143,P,85,856,학원에서 일반인대상으로 캘리그라피
...,...,...,...,...,...
46605,id_0999956,P,85,855,학원에서 학생을 대상으로 영어
46606,id_0999965,P,85,856,"학원에서 일반인 대상으로 캘리그라프,팝아트 등"
46607,id_0999966,P,85,855,학원에서 중고생을 대상으로 수학 교습
46608,id_0999990,P,85,855,공부방에서 학생대상 교과목


## **# 데이터 증분 및 맞춤법 교정**

In [ ]:
# 맞춤법 검사

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-2xt094io
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-2xt094io
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=1d6ebf46487137bfe6fde5f91281d2887ce7d05e516a5d3825d28d07ab856ca3
  Stored in directory: /tmp/pip-ephem-wheel-cache-dblvu9a4/wheels/31/c2/c7/ddbbd9a84ddb0a67c2e52e3324d4b7b47dcf9581af9cd784de
Successfully built py-hanspell


In [ ]:
from hanspell import spell_checker

temp_result=[]
for sentence in df['text']:
  try:
    result=spell_checker.check(sentence)
    temp_result.append(result.checked)
  except:
    temp_result.append(sentence)

df['text']=pd.DataFrame(temp_result)  

In [ ]:
df

In [ ]:
df['digit_3'].value_counts()

856    23521
855    16862
851     3544
852     1333
857      917
853      366
854       67
Name: digit_3, dtype: int64

In [ ]:
# 데이터 증분 코드
import random
import pickle
import re
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)
	return parseText

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break
	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")
	else:
		new_words = ""
	return new_words

def get_synonyms(word):
	synomyms = []
	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass
	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]
	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)	
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""
		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1
	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))
	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))
	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))
	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))
	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))
	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
	augmented_sentences.append(sentence)
	return augmented_sentences

In [ ]:
df1=df[df['digit_3']==851] 
df2=df[df['digit_3']==852] 
df3=df[df['digit_3']==853] 
df4=df[df['digit_3']==854] 
df5=df[df['digit_3']==855] 
df6=df[df['digit_3']==856] 
df7=df[df['digit_3']==857] 

In [ ]:
df1=df1.reset_index(drop=True)
df2=df2.reset_index(drop=True)
df3=df3.reset_index(drop=True)
df4=df4.reset_index(drop=True)
df5=df5.reset_index(drop=True)
df6=df6.reset_index(drop=True)
df7=df7.reset_index(drop=True)

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df1['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=2)
  id_list=['p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n])]
  digit_1_list=[df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n]]
  digit_2_list=[df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n]]
  digit_3_list=[df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df1_to_insert = pd.DataFrame(data_to_insert)
  df1= df1.append(df1_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df1

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000241,P,85,851,초등학교 관할 교육청 인가를 받아 중학교 교육을 위한 서비스
1,id_0000323,P,85,851,초등학교 관할 교육청인가를 받아 초등교육서비스
2,id_0000905,P,85,851,초등학교 관할지 녘 교육청의 인가를 받아 초등 전 학년을 교육
3,id_0001730,P,85,851,유치원 관할 지역교육청의 인가를 받아 취학 전 아동교육
4,id_0002025,P,85,851,초등학교 관할 지역교육청의 인가를 받아 초등교육
...,...,...,...,...,...
14171,pid_0998930,P,85,851,아동교육 관할 지역 전 유치원 취학 받아 교육청의 인가를
14172,pid_0998930,P,85,851,유치원 관할 지역 교육청의 인가를 받아 취학 전 아동교육
14173,pid_0998994,P,85,851,유치원 받아 인가를 지역 관할 교육청의 취학 전 아동교육
14174,pid_0998994,P,85,851,유치원 관할 지역 교육청의 인가를 받아 취학 전 아동교육


In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df2['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=10)
  id_list=['p'+str(df2['AI_id'][n]), 'p'+str(df2['AI_id'][n]), 'p'+str(df2['AI_id'][n]), 'p'+str(df2['AI_id'][n]),'p'+str(df2['AI_id'][n]), 'p'+str(df2['AI_id'][n]), 'p'+str(df2['AI_id'][n]), 'p'+str(df2['AI_id'][n]),'p'+str(df2['AI_id'][n]), 'p'+str(df2['AI_id'][n]), 'p'+str(df2['AI_id'][n])]
  digit_1_list=[df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n],df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n],df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n]]
  digit_2_list=[df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n],df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n],df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n]]
  digit_3_list=[df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n],df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n],df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df2_to_insert = pd.DataFrame(data_to_insert)
  df2= df2.append(df2_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df2

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000406,P,85,852,중학교에서 중학생을 대상으로 고등 진학을 위한 교육서비스
1,id_0000550,P,85,852,인문계 고등학교 인문계 교육 대학 진학을 위한 교육서비스
2,id_0000599,P,85,852,인문계 고등학교 인문계 교육 대학 진학을 위한 교육서비스
3,id_0000711,P,85,852,학교에서 중학생 대상으로 중등교육
4,id_0000816,P,85,852,중학교에서 중학생을 대상으로 고등학교 진학을 위한 교육서비스
...,...,...,...,...,...
15991,pid_0999569,P,85,852,인문계 고등학교 인문계 교육 대학 진학을 위한 교육서비스
15992,pid_0999569,P,85,852,교육 진학을 위한
15993,pid_0999569,P,85,852,고등학교 인문계 진학을 교육 대학 인문계 위한 교육서비스
15994,pid_0999569,P,85,852,인문계 고등학교 인문계 교육 대학 진학을 위한 교육서비스


In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df7['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=16)
  id_list=['p'+str(df7['AI_id'][n]), 'p'+str(df7['AI_id'][n]), 'p'+str(df7['AI_id'][n]), 'p'+str(df7['AI_id'][n]),'p'+str(df7['AI_id'][n]), 'p'+str(df7['AI_id'][n]), 'p'+str(df7['AI_id'][n]), 'p'+str(df7['AI_id'][n]),'p'+str(df7['AI_id'][n]), 'p'+str(df7['AI_id'][n]), 'p'+str(df7['AI_id'][n]), 'p'+str(df7['AI_id'][n]),'p'+str(df7['AI_id'][n]), 'p'+str(df7['AI_id'][n]), 'p'+str(df7['AI_id'][n]), 'p'+str(df7['AI_id'][n]),'p'+str(df7['AI_id'][n])]
  digit_1_list=[df7['digit_1'][n], df7['digit_1'][n], df7['digit_1'][n], df7['digit_1'][n],df7['digit_1'][n], df7['digit_1'][n], df7['digit_1'][n], df7['digit_1'][n],df7['digit_1'][n], df7['digit_1'][n], df7['digit_1'][n], df7['digit_1'][n],df7['digit_1'][n], df7['digit_1'][n], df7['digit_1'][n], df7['digit_1'][n],df7['digit_1'][n]]
  digit_2_list=[df7['digit_2'][n], df7['digit_2'][n], df7['digit_2'][n], df7['digit_2'][n],df7['digit_2'][n], df7['digit_2'][n], df7['digit_2'][n], df7['digit_2'][n],df7['digit_2'][n], df7['digit_2'][n], df7['digit_2'][n], df7['digit_2'][n],df7['digit_2'][n], df7['digit_2'][n], df7['digit_2'][n], df7['digit_2'][n],df7['digit_2'][n]]
  digit_3_list=[df7['digit_3'][n], df7['digit_3'][n], df7['digit_3'][n], df7['digit_3'][n],df7['digit_3'][n], df7['digit_3'][n], df7['digit_3'][n], df7['digit_3'][n],df7['digit_3'][n], df7['digit_3'][n], df7['digit_3'][n], df7['digit_3'][n],df7['digit_3'][n], df7['digit_3'][n], df7['digit_3'][n], df7['digit_3'][n],df7['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df7_to_insert = pd.DataFrame(data_to_insert)
  df7= df7.append(df7_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df7

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000597,P,85,857,사업장에서 방과 후 교사 자격증 교육컨설팅
1,id_0002448,P,85,857,사업장에서 방과 후 위탁사업
2,id_0002831,P,85,857,사업장에서 방과 후 학습 프로그램 교육지원 서비스
3,id_0002971,P,85,857,사무실에서 고객 의뢰로 유학 알선 서비스
4,id_0003648,P,85,857,사무실에서 고객 의뢰로 유학 알선 서비스
...,...,...,...,...,...
16501,pid_0999410,P,85,857,사무실에서 일반인 대상 교육컨설팅
16502,pid_0999410,P,85,857,사무실에서 일반인 대상 교육컨설팅
16503,pid_0999410,P,85,857,사무실에서 일반인 대상 교육컨설팅
16504,pid_0999410,P,85,857,교육컨설팅


In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df3['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=39)
  id_list=['p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n])]
  digit_1_list=[df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n]]
  digit_2_list=[df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n]]
  digit_3_list=[df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df3_to_insert = pd.DataFrame(data_to_insert)
  df3= df3.append(df3_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df3

,AI_id,digit_1,digit_2,digit_3,text
0,id_0053872,P,85,853,재능대학 기술 및 직업훈련 교육 전문대학
1,id_0094367,P,85,853,영상대학교 고등전문교육 전문대학
2,id_0197455,P,85,853,"사업장에서 일반 대상인들을 모집, 학습관, 대학"
3,id_0201191,P,85,853,국제법률대학원 대학과정을 수료한 전문화된 교과과정 교육
4,id_0222859,P,85,853,국책 사어 연구활동 다문화연구
...,...,...,...,...,...
15001,pid_0981582,P,85,853,보건대학원에서 학생들을 대상으로 의료 교육 제공
15002,pid_0981582,P,85,853,보건대학원에서 학생들을 대상으로 의료 교육 제공
15003,pid_0981582,P,85,853,보건대학원에서 학생들을 대상으로 의료 교육 제공
15004,pid_0981582,P,85,853,보건대학원에서 학생들을 대상으로 의료 교육 제공


In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df4['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=79)
  id_list=['p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n]),'p'+str(df4['AI_id'][n])]
  digit_1_list=[df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n],df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n],df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n],df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n],df4['digit_1'][n],df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n],df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n],df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n],df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n],df4['digit_1'][n],df4['digit_1'][n],df4['digit_1'][n],df4['digit_1'][n],df4['digit_1'][n],df4['digit_1'][n],df4['digit_1'][n],df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n],df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n],df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n],df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n],df4['digit_1'][n],df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n],df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n],df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n],df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n],df4['digit_1'][n],df4['digit_1'][n],df4['digit_1'][n],df4['digit_1'][n],df4['digit_1'][n],df4['digit_1'][n],df4['digit_1'][n]]
  digit_2_list=[df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n],df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n],df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n],df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n],df4['digit_2'][n],df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n],df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n],df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n],df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n],df4['digit_2'][n],df4['digit_2'][n],df4['digit_2'][n],df4['digit_2'][n],df4['digit_2'][n],df4['digit_2'][n],df4['digit_2'][n],df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n],df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n],df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n],df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n],df4['digit_2'][n],df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n],df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n],df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n],df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n],df4['digit_2'][n],df4['digit_2'][n],df4['digit_2'][n],df4['digit_2'][n],df4['digit_2'][n],df4['digit_2'][n],df4['digit_2'][n]]
  digit_3_list=[df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n],df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n],df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n],df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n],df4['digit_3'][n],df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n],df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n],df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n],df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n],df4['digit_3'][n],df4['digit_3'][n],df4['digit_3'][n],df4['digit_3'][n],df4['digit_3'][n],df4['digit_3'][n],df4['digit_3'][n],df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n],df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n],df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n],df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n],df4['digit_3'][n],df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n],df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n],df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n],df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n],df4['digit_3'][n],df4['digit_3'][n],df4['digit_3'][n],df4['digit_3'][n],df4['digit_3'][n],df4['digit_3'][n],df4['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df4_to_insert = pd.DataFrame(data_to_insert)
  df4= df4.append(df4_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df4

,AI_id,digit_1,digit_2,digit_3,text
0,id_0084262,P,85,854,전북지역 맹아들의 교육 서비스
1,id_0215335,P,85,854,국제 학교에서 외국인 초중고를 대상으로 교육서비스
2,id_0215913,P,85,854,국제 학교에서 외국인 학생 대상으로 교육을 가르침(중. 고등학교)
3,id_0216751,P,85,854,국제 학교에서 주한 외국인 초. 중. 고대상으로 교육서비스
4,id_0231633,P,85,854,서울시 교육청 인가 대안학교에서 학생에게 예술통합 대안교육
...,...,...,...,...,...
5422,pid_0972754,P,85,854,"학교에서 중, 고등학교까지 대안교육"
5423,pid_0972754,P,85,854,"고등학교까지 중, 대안교육 학교에서"
5424,pid_0972754,P,85,854,"학교에서 중, 고등학교까지 대안교육"
5425,pid_0972754,P,85,854,"대안교육 학교에서 고등학교까지 중,"


기존 데이터 합치기

In [ ]:
df=pd.concat([df1, df2,df3,df4,df5,df6,df7])

In [ ]:
df['digit_3'].value_counts()

856    23521
855    16862
857    16506
852    15996
853    15006
851    14176
854     5427
Name: digit_3, dtype: int64

In [ ]:
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000241,P,85,851,초등학교 관할 교육청 인가를 받아 중학교 교육을 위한 서비스
1,id_0000323,P,85,851,초등학교 관할 교육청인가를 받아 초등교육서비스
2,id_0000905,P,85,851,초등학교 관할지 녘 교육청의 인가를 받아 초등 전 학년을 교육
3,id_0001730,P,85,851,유치원 관할 지역교육청의 인가를 받아 취학 전 아동교육
4,id_0002025,P,85,851,초등학교 관할 지역교육청의 인가를 받아 초등교육
...,...,...,...,...,...
16501,pid_0999410,P,85,857,사무실에서 일반인 대상 교육컨설팅
16502,pid_0999410,P,85,857,사무실에서 일반인 대상 교육컨설팅
16503,pid_0999410,P,85,857,사무실에서 일반인 대상 교육컨설팅
16504,pid_0999410,P,85,857,교육컨설팅


In [ ]:
df.to_csv('df_P_inc.csv', index=False, encoding='utf-8-sig')